In [1]:
# NEED TO RESTART KERNEL TO LOAD CHANGES TO IMPORTS :(

import os
import sys

# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath("./../src"))

import datetime
import importlib

import matplotlib.pyplot as plt

import data_cache
import data_loader
import field_models
import plot_tools
import psd_tool
from energy_channels import EnergyChannel

importlib.reload(plot_tools)
importlib.reload(field_models)
importlib.reload(psd_tool)
importlib.reload(data_loader)
importlib.reload(data_cache)

%matplotlib qt

# NEED TO RESTART KERNEL TO LOAD CHANGES TO IMPORTS :(

This unreleased version of SpacePy is not supported by the SpacePy team.


In [2]:
start = datetime.datetime(year=2015, month=1, day=1)
end = datetime.datetime(year=2015, month=1, day=7)
sat = "A"

In [3]:
REPT = data_loader.load_compressed_rept_data(satellite=sat, start=start, end=end)

fig, axs = plt.subplots(3, 1, sharex=True)
fig.set_size_inches(16, 9)

E1 = EnergyChannel.MeV_1_8
E2 = EnergyChannel.MeV_2_1
E3 = EnergyChannel.MeV_2_6

axs[0].set_xlim(start, end)
axs[1].set_xlim(start, end)
axs[2].set_xlim(start, end)

plot_tools.plot_l_vs_time_log_colors(
    x=REPT["EPOCH"],
    y=REPT["L"],
    c=REPT["FESA"][:, E1.value],
    cbar_label="($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)\n",
    axis=axs[0],
)
plot_tools.plot_l_vs_time_log_colors(
    x=REPT["EPOCH"],
    y=REPT["L"],
    c=REPT["FESA"][:, E2.value],
    cbar_label="($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)\n",
    axis=axs[1],
)
plot_tools.plot_l_vs_time_log_colors(
    x=REPT["EPOCH"],
    y=REPT["L"],
    c=REPT["FESA"][:, E3.value],
    cbar_label="($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)\n",
    axis=axs[2],
)

axs[0].set_ylim(3, 6)
axs[1].set_ylim(3, 6)
axs[2].set_ylim(3, 6)


axs[0].set_title(f"{REPT["ENERGIES"][E1.value]} MeV", fontsize=10)
axs[1].set_title(f"{REPT["ENERGIES"][E2.value]} MeV", fontsize=10)
axs[2].set_title(f"{REPT["ENERGIES"][E3.value]} MeV", fontsize=10)

axs[0].set_ylabel("L")
axs[1].set_ylabel("L")
axs[2].set_ylabel("L")

mid_for_title = (fig.subplotpars.right + fig.subplotpars.left) / 2
fig.suptitle(f"RBSP-{sat}, Spin Averaged Electron Flux", fontsize=14, x=mid_for_title)
plt.show()

Loading REPT data between: 2015-01-01 00:00:00 and 2015-01-07 00:00:00.
Loading : REPT_201501_A.npz


In [4]:
field_model = field_models.model.TS04D

mu_vals = [500, 1000, 3000]
chosen_k = 0.07

In [5]:
psd_refs = data_loader.load_psd(satellite=sat, field_model=field_model, start=start, end=end)

2015-01-01 00:00:00
Loading : PSD_201501_A_TS04D.npz


In [6]:
for chosen_mu in mu_vals:

    print(f"Selecting PSD for A at mu = {chosen_mu}, k = {chosen_k}")

    selected_psd_refs = psd_tool.select_mu_and_k_from_psd(
        refs=psd_refs, chosen_mu=chosen_mu, chosen_k=chosen_k
    )

    data_cache.cache_psd_at_selected_mu_and_k(
        selected_psd_refs,
        satellite=sat,
        start=start,
        end=end,
        mu=chosen_mu,
        k=chosen_k,
        field_model=field_model,
    )

Selecting PSD for A at mu = 500, k = 0.07
Time taken for loop: 2.245243787765503
Selecting PSD for A at mu = 1000, k = 0.07
Time taken for loop: 1.053934097290039
Selecting PSD for A at mu = 3000, k = 0.07
Time taken for loop: 1.1295132637023926


In [7]:
Lstar_min = 3.0
Lstar_max = 6.0

fig, axs = plt.subplots(len(mu_vals), 1, sharex=True)
fig.set_size_inches(16, 9)

axs[0].set_xlim(start, end)

for k, chosen_mu in enumerate(mu_vals):

    selected_psd_refs = data_cache.load_psd_cache(
        satellite=sat, start=start, end=end, mu=chosen_mu, k=chosen_k, field_model=field_model
    )

    epoch = selected_psd_refs["EPOCH"]
    Lstar = selected_psd_refs["L_STAR"]
    PSD = selected_psd_refs["PSD"]

    outer_belt = (3 < Lstar) & (Lstar < 7)

    plot_tools.plot_l_vs_time_log_colors(
        x=epoch[outer_belt],
        y=Lstar[outer_belt],
        c=PSD[outer_belt],
        cbar_label=r"PSD $(c/MeV/cm)^{3}$",
        axis=axs[k],
    )
    axs[k].set_title(
        rf"RBSP-{sat}, K = {chosen_k}, ($R_E$$G^{{1/2}}$), $\mu$ = {chosen_mu} (MeV / G), {field_model.name}"
    )
    axs[k].set_ylabel("L*")
    axs[k].set_ylim(Lstar_min, Lstar_max)
    axs[k].set_aspect("auto")

mid_for_title = (fig.subplotpars.right + fig.subplotpars.left) / 2
fig.suptitle(
    f"RBSP-{sat}, Phase Space Density", fontsize=14, x=mid_for_title, verticalalignment="top"
)
plt.show()